In [1]:
# Импорт библиотек
import sqlite3 
from sqlite3 import Error 
import pandas as pd

**Шаг 1**  
(Балл - 2) Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:  

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции  
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции  
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента  
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала  
(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции  

Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:  

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента  
(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте  
(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте  
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента  
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента  

Примечание для следующих шагов: сумма транзакций не может быть null-ом, это всегда число.

In [2]:
# Функция создания Connection с БД
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

In [3]:
# Создание Connection с БД. Неявно создает БД, если не существует.
con = create_connection("HW3.db")

Connection to SQLite DB successful


In [4]:
# Функция выполнения запроса к БД. Создание Cursor, выполнение запроса/ошибка, если запрос выполнен, то 
# коммит Connection, закрытие Cursor.
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
        cursor.close()
    except Error as e:
        print(f"The error '{e}' occurred")

In [5]:
# Запрос на создание таблицы transaction_bd в БД.
create_transaction_table = """
CREATE TABLE IF NOT EXISTS transaction_bd (
  TRANSACTION_ID INTEGER NOT NULL,
  TX_DATETIME TEXT NOT NULL,
  CUSTOMER_ID INTEGER NOT NULL,
  TERMINAL_ID INTEGER NOT NULL,
  TX_AMOUNT REAL NOT NULL
);
"""

In [6]:
# Исполнение запроса на создание таблицы transaction_bd в БД.
execute_query(con, create_transaction_table)

Query executed successfully


In [7]:
# Запрос на создание таблицы customer_bd в БД.
create_customer_table = """
CREATE TABLE IF NOT EXISTS customer_bd (
  CLIENT_ID INTEGER NOT NULL,
  START_DT TEXT NOT NULL,
  END_DT TEXT NOT NULL,
  CLIENT_NAME TEXT NOT NULL,
  YEAR_BIRTH TEXT NOT NULL
);
"""

In [8]:
# Исполнение запроса на создание таблицы customer_bd в БД.
execute_query(con, create_customer_table)

Query executed successfully


In [9]:
# Чтение файла csv в датафрейм.
df_trans = pd.read_csv('transactions_for_dz2.csv')
df_trans

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
...,...,...,...,...,...
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44


In [10]:
# Кол-во уникальных клиентов в датафрейме/таблице.
df_trans['CUSTOMER_ID'].nunique()

4986

In [11]:
# Добавление всех записей с файла transactions_for_dz2.csv в таблицу transaction_bd
df_trans.to_sql('transaction_bd', con, if_exists='append', index=False)

In [12]:
# Чтение файла csv в датафрейм.
df_cust = pd.read_csv('client_info.csv', sep=';')
df_cust

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


In [13]:
# Информация о действующих клиентах.
df_cust_active = df_cust[df_cust['END_DT'] >= '2023-05-01']
df_cust_active

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
27,2010-01-01,2999-12-31,Hazel,1945,1172
28,2010-01-01,2999-12-31,Madison,1945,2912
29,2010-01-01,2999-12-31,Ellie,1945,3604
30,2010-01-01,2999-12-31,Lily,1946,4361
31,2010-02-01,2999-12-31,Nova,1946,3700
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


In [14]:
# Транзакции действующих клиентов.
trans_active = df_trans[df_trans['CUSTOMER_ID'].isin(set(df_cust_active['CLIENT_ID']))]
trans_active

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
5,5,2023-01-01 00:10:45,568,8803,401.17
6,6,2023-01-01 00:11:30,2803,5490,938.54
...,...,...,...,...,...
1048568,1048568,2023-04-20 10:07:01,3670,8244,616.24
1048569,1048569,2023-04-20 10:07:05,1534,4927,80.99
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38


In [15]:
# ID действующих клиентов, которые не совершали транзакции за период анализа.
set(df_cust_active['CLIENT_ID']) - set(trans_active['CUSTOMER_ID'])

{11111, 77777}

In [16]:
# Информация о действующих клиентах, которые не совершали транзакции за период анализа.
df_cust_active[df_cust_active['CLIENT_ID'].isin(['11111', '77777'])]

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
4918,2023-03-01,2999-12-31,Sofia,1999,77777
4919,2021-11-01,2999-12-31,Oksi,2000,11111


In [17]:
# Добавление всех записей с файла client_info.csv в таблицу customer_bd
df_cust.to_sql('customer_bd', con, if_exists='append', index=False)

**Шаг 2**  
(Балл - 1 за каждый пункт) Написать скрипты:  

**2a.** Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [18]:
# Запрос для пункта 2a
query_a = """
WITH cust_2a AS (
SELECT CUSTOMER_ID, CLIENT_NAME, SUM(TX_AMOUNT) AS SUM_TRANSACTION
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY CUSTOMER_ID
)
SELECT CLIENT_NAME
FROM cust_2a
WHERE SUM_TRANSACTION = (SELECT MAX(SUM_TRANSACTION) FROM cust_2a);
"""

In [19]:
# Выполнение запроса для пункта 2a и запись в датафрейм.
df_a = pd.read_sql_query(query_a, con)
df_a

,CLIENT_NAME
0,Kylee


**2b.** Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [20]:
# Запрос для пункта 2b
query_b = """
WITH cust_2b AS (
SELECT CUSTOMER_ID, CLIENT_NAME, SUM(TX_AMOUNT) AS SUM_TRANSACTION
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY CUSTOMER_ID
)
SELECT CLIENT_NAME
FROM cust_2b
WHERE SUM_TRANSACTION = (SELECT MIN(SUM_TRANSACTION) FROM cust_2b);
"""

In [21]:
# Выполнение запроса для пункта 2b и запись в датафрейм.
df_b = pd.read_sql_query(query_b, con)
df_b

,CLIENT_NAME
0,Kloe


**2c.** Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.

In [22]:
# Запрос для пункта 2c
query_c = """
SELECT CLIENT_ID, SUM(TX_AMOUNT) AS SUM_TRANSACTION
FROM customer_bd
LEFT JOIN transaction_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01' AND CLIENT_ID LIKE '111%' AND strftime('%Y-%m-%d', TX_DATETIME) < '2023-04-01'
GROUP BY CLIENT_ID
;
"""

In [23]:
# Выполнение запроса для пункта 2c и запись в датафрейм.
df_c = pd.read_sql_query(query_c, con)
df_c

,CLIENT_ID,SUM_TRANSACTION
0,1112,35273.16
1,1113,229558.85
2,1114,108622.52
3,1115,38690.76
4,1116,191203.59
5,1117,52751.58
6,1118,353003.39
7,1119,221192.71


**2d.** Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.

In [24]:
# Запрос для пункта 2d
query_d = """
SELECT YEAR_BIRTH, SUM(TX_AMOUNT) AS SUM_TRANSACTION
FROM customer_bd
LEFT JOIN transaction_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY YEAR_BIRTH
ORDER BY YEAR_BIRTH DESC
;
"""

In [25]:
# Выполнение запроса для пункта 2d и запись в датафрейм.
df_d = pd.read_sql_query(query_d, con)
df_d

,YEAR_BIRTH,SUM_TRANSACTION
0,2000,8.632720e+07
1,1999,1.041507e+06
2,1998,6.016750e+05
3,1997,8.479943e+06
4,1996,1.483253e+06
5,1995,6.321200e+06
6,1994,6.244281e+06
7,1993,5.732058e+06
8,1992,5.251433e+06
9,1991,7.141074e+06


**2e.** Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [26]:
# Запрос для пункта 2e
query_e = """
SELECT CLIENT_ID, COUNT(TX_AMOUNT) AS COUNT_TRANSACTION
FROM customer_bd
LEFT JOIN transaction_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY CLIENT_ID
ORDER BY COUNT_TRANSACTION DESC
;
"""

In [27]:
# Выполнение запроса для пункта 2e и запись в датафрейм.
df_e = pd.read_sql_query(query_e, con)
df_e

,CLIENT_ID,COUNT_TRANSACTION
0,2891,477
1,2580,464
2,1953,455
3,2932,454
4,3864,453
...,...,...
3733,1880,1
3734,1095,1
3735,822,1
3736,77777,0


**Шаг 3**  
(Балл - 3) Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [28]:
# Запрос для шага 3
query_3 = """
SELECT 
CASE strftime('%m', TX_DATETIME)
    WHEN '01' THEN 'Январь'
    WHEN '02' THEN 'Февраль'
    WHEN '03' THEN 'Март'
    WHEN '04' THEN 'Апрель'
END AS MONTH,
SUM(TX_AMOUNT) AS SUM_TRANSACTION
FROM transaction_bd
LEFT JOIN customer_bd ON CUSTOMER_ID = CLIENT_ID
WHERE END_DT >= '2023-05-01'
GROUP BY MONTH
ORDER BY MONTH DESC
;
"""

In [29]:
# Выполнение запроса для шага 3 и запись в датафрейм.
df_3 = pd.read_sql_query(query_3, con)
df_3

,MONTH,SUM_TRANSACTION
0,Январь,1.159199e+08
1,Февраль,1.062295e+08
2,Март,1.150534e+08
3,Апрель,7.261812e+07


In [30]:
# Закрытие Connection с БД.
con.close()